## Импорты

In [2]:
import os, torch, mlflow

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    # get_peft_model,
)

from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format, DataCollatorForCompletionOnlyLM
from dataclasses import dataclass
import random
import numpy as np
import os
import re
import json
import pandas as pd
import traceback
import torch
from GPUtil import showUtilization as gpu_usage
from peft import get_peft_model


## Конфигурация, переменные и сервисные функции

In [3]:
@dataclass
class Config:
    model_name = "unsloth/Llama-3.2-1B-Instruct"
    new_model = "llama-3.1-8b-chat-house"
    torch_dtype = torch.float16
    attn_implementation = "eager"
cfg = Config()

# Очистка GPU
def clear_gpu_memory():
    print("\nInitial GPU Usage")
    gpu_usage()  # Показывает текущее использование GPU

    torch.cuda.empty_cache()  # Очищает кеш CUDA

    print("\nGPU Usage after emptying the cache")
    gpu_usage()  # Показывает использование GPU после очистки кеша

## Подготовка модели

In [4]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=cfg.torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# Загрузка модели и токенизатора
model = AutoModelForCausalLM.from_pretrained(
    cfg.model_name,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=cfg.attn_implementation
)
tokenizer = AutoTokenizer.from_pretrained(cfg.model_name)

# Явный сброс существующего чат-шаблона перед настройкой
if tokenizer.chat_template is not None:
    tokenizer.chat_template = None

# 1. Настройка формата чата (теперь можно безопасно добавлять)
model, tokenizer = setup_chat_format(model, tokenizer)

# 2. Добавляем pad_token после настройки чата
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '<|pad|>'})

# 3. Синхронизация размеров ПОСЛЕ ВСЕХ изменений
model.resize_token_embeddings(len(tokenizer))

# Дополнительные настройки
tokenizer.padding_side = 'right'  # Для корректной работы с пакетами

# Проверка размеров
print(f"[Проверка] Размер словаря: {len(tokenizer)}")
print(f"[Проверка] Размер эмбеддингов: {model.get_input_embeddings().weight.shape[0]}")

# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

# Применяем LoRA
model = get_peft_model(model, peft_config)

# Загрузка данных
dataset_dict = load_dataset("json", data_files="../data/processed/context_answer.json")
dataset = dataset_dict["train"] if "train" in dataset_dict else dataset_dict

# Форматирование данных
def format_chat_template(row):
    row_json = [{"role": "user", "content": row["q"]},
               {"role": "assistant", "content": row["a"]}]
    text = tokenizer.apply_chat_template(row_json, tokenize=False)
    return text

# Создание data collator
response_template = "assistant\n"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

# Разделение данных
dataset_sh = dataset.shuffle(seed=2024).select(range(len(dataset)))
dataset_sh = dataset_sh.train_test_split(0.1)

# Настройка аргументов обучения
training_arguments = TrainingArguments(
    output_dir=cfg.new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    eval_strategy="steps",
    eval_steps=50,
    logging_steps=100,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=True,
    group_by_length=True,
    logging_dir='../logs',
    report_to=["mlflow"],
    run_name="Llama-3.2-house",
)

# Создание тренера
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_sh["train"],
    eval_dataset=dataset_sh["test"],
    peft_config=peft_config,
    tokenizer=tokenizer,
    formatting_func=format_chat_template,
    args=training_arguments,
    data_collator=collator,
)


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


[Проверка] Размер словаря: 128258
[Проверка] Размер эмбеддингов: 128258


Generating train split: 0 examples [00:00, ? examples/s]

/tmp/ipykernel_54364/2497587746.py:94: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Applying formatting function to train dataset:   0%|          | 0/11096 [00:00<?, ? examples/s]

...

Converting eval dataset to ChatML:   0%|          | 0/1233 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/1233 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/1233 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/1233 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


## Процесс обучения

In [5]:
# Очистка памяти GPU перед обучением
clear_gpu_memory()

# Обучение
try:
    trainer.train()
except Exception as e:
    print("Произошла ошибка:")
    traceback.print_exc()
    print(f"Тип ошибки: {type(e).__name__}")
    print(f"Сообщение ошибки: {str(e)}")

model.resize_token_embeddings(len(tokenizer))


Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 20% |

GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  0% |  9% |


Step,Training Loss,Validation Loss
50,No log,3.468130
100,3.450800,3.343053
150,3.450800,3.402377
200,3.383300,3.353297
250,3.383300,3.313370
300,3.250500,3.314968
350,3.250500,3.311322
400,3.310000,3.271931
450,3.310000,3.285639
500,3.322800,3.270088


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

Размер словаря перед сохранением: 128258
Размер эмбеддингов модели перед сохранением: 128258
Размер словаря перед сохранением: 128258
Размер эмбеддингов модели перед сохранением: 128258


## Сохранение модели для дальнейшего использования и тестирования

Сохраняем отдельно:
- Токенизатор с новыми токенами
- Адаптеры LoRA
- Конфигурацию модели

In [6]:
path_to_save = "../models/Llama-finetuned"

# Убедимся, что размеры синхронизированы перед сохранением
model.resize_token_embeddings(len(tokenizer))

# Сохраняем токенизатор
tokenizer.save_pretrained(path_to_save)

# Сохраняем адаптеры LoRA отдельно
model.save_pretrained(path_to_save) 

# Для загрузки и использования:
def load_custom_model(model_path, base_model_name):
    # Загружаем базовую модель
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        quantization_config=bnb_config,
        device_map="auto"
    )
    
    # Синхронизируем с токенизатором
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    base_model.resize_token_embeddings(len(tokenizer))
    
    # Загружаем адаптеры
    model = PeftModel.from_pretrained(base_model, model_path)
    
    return model, tokenizer

# Загрузка модели
loaded_model, loaded_tokenizer = load_custom_model(path_to_save, cfg.model_name)

# Проверка размеров
print(f"Tokenizer size: {len(loaded_tokenizer)}")
print(f"Model embeddings: {loaded_model.get_input_embeddings().weight.shape[0]}")

# Объединение для инференса
merged_model = loaded_model.merge_and_unload()
merged_model.save_pretrained("../models/Llama-merged")

Final tokenizer size: 128258
Final model embeddings size: 128258


/opt/conda/lib/python3.11/site-packages/peft/utils/save_and_load.py:241: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


Tokenizer size: 128258
Model embeddings: 128258


/opt/conda/lib/python3.11/site-packages/peft/tuners/lora/bnb.py:351: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
